In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from finta import TA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report



### Initial Data Import and Cleaning

In [ ]:
# Read the csv file 
amzn_df = pd.read_csv(Path("Resources/amzn.csv"))

# Convert 'time' column from timestamp (seconds since epoch) to acutal time
amzn_df ['time'] = pd.to_datetime(
    amzn_df['time'],
    unit = 's',
    infer_datetime_format=True,
    utc=True
)    
# Convert timezone from UTC to Eastern Time
amzn_df['time'] = amzn_df['time'].dt.tz_convert('US/Eastern')

# Set 'time' column as the index 
amzn_df.set_index('time', inplace=True)

# Review df
amzn_df.head()          

In [ ]:
# Filter column and only keep ones needed
amzn_df = amzn_df[['open','high','low','close','VWAP','Volume','Volume MA','EMA']]

#Rename EMA to 200EMA
amzn_df = amzn_df.rename(columns={'EMA':'200EMA'})

#Dropping NAN Values 
amzn_df = amzn_df.dropna()

# Review the DataFrame
amzn_df.head()

In [ ]:
# Read the csv file 
meta_df = pd.read_csv(Path("Resources/meta.csv"))

# Convert 'time' column from timestamp (seconds since epoch) to acutal time
meta_df ['time'] = pd.to_datetime(
    meta_df['time'],
    unit = 's',
    infer_datetime_format=True,
    utc=True
)    
# Convert timezone from UTC to Eastern Time
meta_df['time'] = meta_df['time'].dt.tz_convert('US/Eastern')

# Set 'time' column as the index 
meta_df.set_index('time', inplace=True)

# Review df
meta_df.head()

In [ ]:
# Filter column and only keep ones needed
meta_df = meta_df[['open','high','low','close','VWAP','Volume','Volume MA','EMA']]

#Rename EMA to 200EMA
meta_df = meta_df.rename(columns={'EMA':'200EMA'})

#Dropping NAN Values 
meta_df = meta_df.dropna()

# Review the DataFrame
meta_df.head()

In [ ]:
# Read the csv file 
tsla_df = pd.read_csv(Path("./Resources/tsla.csv"))

# Convert 'time' column from timestamp (seconds since epoch) to acutal time
tsla_df ['time'] = pd.to_datetime(
    tsla_df['time'],
    unit = 's',
    infer_datetime_format=True,
    utc=True
)    
# Convert timezone from UTC to Eastern Time
tsla_df['time'] = tsla_df['time'].dt.tz_convert('US/Eastern')

# Set 'time' column as the index 
tsla_df.set_index('time', inplace=True)
                      
# Review df
tsla_df.head()   

In [ ]:
# Filter column and only keep ones needed
tsla_df = tsla_df[['open','high','low','close','VWAP','Volume','Volume MA','EMA']]

#Rename EMA to 200EMA
tsla_df = tsla_df.rename(columns={'EMA':'200EMA'})

#Dropping NAN Values 
tsla_df = tsla_df.dropna()

# Review the DataFrame
tsla_df.head()

In [ ]:
# Read the csv file 
vix_df = pd.read_csv(Path("Resources/vix.csv"))

# Convert 'time' column from timestamp (seconds since epoch) to acutal time
vix_df ['time'] = pd.to_datetime(
    vix_df['time'],
    unit = 's',
    infer_datetime_format=True,
    utc=True
)    
# Convert timezone from UTC to Eastern Time
vix_df['time'] = vix_df['time'].dt.tz_convert('US/Eastern')

# Set 'time' column as the index 
vix_df.set_index('time', inplace=True)

# Calculate VIX EMA 
vix_df['VIX_40_EMA'] = TA.EMA(vix_df,40)

#Remove unnecessary columns
vix_df = vix_df[['close','VIX_40_EMA']]

#Rename close column
vix_df = vix_df.rename(columns={'close':'VIX_close'})

# Review df
vix_df.head()    

In [ ]:
# Add VIX data to stock dfs
AMZN = pd.concat([amzn_df,vix_df],axis=1)
META = pd.concat([meta_df,vix_df],axis=1)
TSLA = pd.concat([tsla_df,vix_df],axis=1)


In [ ]:
AMZN

### Trading Algo (Signal Generation)

In [ ]:
#set some options for display and troubleshooting
pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.width", 1000)

In [ ]:
# Define periods for fast and slow EMAs (triggers)
fast_ema = 9
slow_ema = 40

# Initialize list of ticker dfs for use in for loop
ticker_df = [AMZN,META,TSLA]

# For each ticker dataframe in the list:
for df in ticker_df:
    
    # Calculate % returns for later use
    df['pct_returns'] = df['close'].pct_change()
    
    # Calculate Fast EMA 
    df['Fast_EMA'] = TA.EMA(df,fast_ema)
    
    # Calculate Slow EMA
    df['Slow_EMA'] = TA.EMA(df,slow_ema)

    # Initialize empty Signal column with 0.0 default value
    df['Signal'] = 0.0

    # Generate Signal value ("long if fast_ema > slow_ema", otherwise flip short)
    df['Signal'] = np.where(
        df['Slow_EMA'] < df['Fast_EMA'], 1.0, -1.0)
    
#This loop should not generate Entry/Exit - this will not be needed until after ML predictions    
    # Create Entry/Exit column and fill with the differences (trades) from Signal column
    #df['Entry/Exit'] = df['Signal'].diff()
    
    #Drop any NA values
    df.dropna()


In [ ]:
# Define features set
X = amzn_df[['open', 'high', 'low', 'close']]
X.head()


In [ ]:
# Define target vector
y = amzn_df["'open', 'high', 'low', 'close'"].ravel()
y[:5]

In [ ]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [ ]:
# Creating StandardScaler instance
scaler = StandardScaler()


In [ ]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)


In [ ]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


### Adding Additional Features to Time-Filtered Datasets

## Fitting the Random Forest Model

In [ ]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [ ]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

## Making Predictions Using the Decsion Tree Model

In [ ]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)


## Model Evaluation

In [ ]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)


In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))


## Feature Importance

In [ ]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf_model.feature_importances_


In [ ]:
# We can sort the features by their importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

In [ ]:
# Filter each dataframe to 10yr period
amzn_df_filt = AMZN.loc['05-2012':'05-2022']
meta_df_filt = META.loc['05-2012':'05-2022']
tsla_df_filt = TSLA.loc['05-2012':'05-2022']

In [ ]:
tsla_df_filt.head(3)